# Interactive 3D visualization of Fastscape model outputs

Model outputs can be viewed interactively in 3-d rendered scenes using [ipyfastscape](https://github.com/fastscape-lem/ipyfastscape)'s `TopoViz3d`, very much like [Paraview](https://www.paraview.org/) but in Jupyter notebooks.

In [1]:
import numpy as np
import xsimlab as xs
import xarray as xr

from fastscape.models import basic_model
from fastscape.models import sediment_model

from ipyfastscape import TopoViz3d

In [2]:
from fastscape.processes import Escarpment, TwoBlocksUplift

model = (sediment_model
         .update_processes({'init_topography': Escarpment,
                                   'uplift': TwoBlocksUplift
}))

Let's start by setting and running a simple fastscape simulation. Model outputs will be saved along the `time` dimension (clock).

In [46]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 4e5 + 2e3, 2e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 201],
        'grid__length': [1e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'core'],
        'init_topography': {
            'x_left': 1e4,
            'x_right': 1e4,
            'elevation_left': 1e2,
            'elevation_right': 2e2
        },
        'uplift': {
            'x_position': 1e4,
            'rate_left': 5e-4,
            'rate_right': 1e-3
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-4,
            'k_coef_soil': 1.5e-4,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1.5e-2
        }
    },
    output_vars={
        'topography__elevation': 'time',
        'erosion__rate': 'time'
    }
)

In [47]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

             0% | initialize 

The output dataset `out_ds` can be readily used with `TopoViz3d`, which creates a graphical interface with the 3-d rendered scene as well as convenient controls for some display properties. Here we also explicitly set `time="time"` to enable animation of the 3D scene using some additional controls.

In [48]:
app = TopoViz3d(out_ds, canvas_height=600, time_dim="time")

app.show()

Output(layout=Layout(height='640px'))

It is also possible to control the visualization from the notebook code cells. For example, let's show the time step that is the closest to a given time value (you should see the output of the cell here above updated accordingly):

In [ ]:
app.components['timestepper'].go_to_time(1.15e6)